In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Step 1: Load model and tokenizer
model_name = "Qwen/Qwen2.5-3B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).cuda()
model.eval()


/project/uvadm/zhenyu/miniconda3/envs/mechanistic/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 2 files: 100%|██████████| 2/2 [00:06<00:00,  3.22s/it]
Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.84s/it]


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-35): 36 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear(in_features=2048, out_features=256, bias=True)
          (v_proj): Linear(in_features=2048, out_features=256, bias=True)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (up_proj): Linear(in_features=2048, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((2048,), eps=1e-06)
    (rotary_emb):

In [ ]:

# Step 2: Define your prompt
prompt = "Marine biologists recently explored the Mariana Trench, the deepest part of the world's oceans. They did not discover any new species of deep sea fish on this expedition. The deepest waters they sampled did not contain any fish species capable of surviving in waters below 0\u00b0C.\nIf a new species of deep sea fish adapted to the extreme conditions was discovered in the Mariana Trench or a fish species that can survive below 0\u00b0C was found, it would indicate that life has adapted to the harsh conditions in the trench.\nBased on the expedition findings, can we conclude whether the Mariana Trench contains life that has adapted to the extreme conditions there?"
prompt += " (Directly provide answer in True, False, or N/A (Neither)). Answer:"

# Step 3: Tokenize
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# Step 4: Generate output
with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=50,
        do_sample=False,       # greedy decoding
        pad_token_id=tokenizer.eos_token_id
    )

# Step 5: Decode and print
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

Marine biologists recently explored the Mariana Trench, the deepest part of the world's oceans. They did not discover any new species of deep sea fish on this expedition. The deepest waters they sampled did not contain any fish species capable of surviving in waters below 0°C.
If a new species of deep sea fish adapted to the extreme conditions was discovered in the Mariana Trench or a fish species that can survive below 0°C was found, it would indicate that life has adapted to the harsh conditions in the trench.
Based on the expedition findings, can we conclude whether the Mariana Trench contains life that has adapted to the extreme conditions there? (Directly provide answer in True, False, or N/A (Neither)). Answer: N/A
The given information does not provide direct evidence about whether life has adapted to the extreme conditions in the Mariana Trench. The expedition did not find any new species of deep sea fish, nor did it find any fish species capable of surviving
